In [1]:
import numpy as np
import pandas as pd

## Conjuntos 
Del modelo de optimización

| **Término**  | **Descripción**                |
|--------------|--------------------------------|
| $cacopios$   | Conjunto de centros de acopio  |
| $clientes$   | Conjunto de clientes           |
| $productos$  | Conjunto de productos          |

In [2]:
cacopios = 'CAcopios'
clientes = 'Clientes'
productos = 'Productos'

In [3]:
csv_base = './csv/'

routes = {
    # Capacidades de centros de acopio
    'cic': 'CapInvCA.csv',
    'cictest': 'CapInvCAtest.csv',
    # Costo de inventario del centro de acopio
    'cia' : 'CostoInvCA.csv',
    'ciatest' : 'CostoInvCAtest.csv',
    # Costo de transporte del centro de acopio al cliente
    'ctac' : 'CostoTransCAClie.csv',
    'ctactest' : 'CostoTransCAClietest.csv'
}

csv_config = {
    'delimiter' : ';',
    'decimal' : ','
}

## Centros de acopio

In [4]:
# cacopios_df = pd.read_csv(csv_base + cacopios + '.csv')
cacopios_df = pd.read_csv(csv_base + cacopios + 'test' + '.csv')
cacopios_df

,CAcopios
0,San Juan del Cesar
1,Riohacha
2,Maicao


## Clientes

In [5]:
# clientes_df = pd.read_csv(csv_base + clientes + '.csv')
clientes_df = pd.read_csv(csv_base + clientes + 'test' + '.csv')
clientes_df

,Clientes
0,Distribuidor1
1,Distribuidor2


## Productos

In [6]:
productos_df = pd.read_csv(csv_base + productos + '.csv')
productos_df.head()

,Productos
0,Queso duro
1,Queso blando


### Capacidades 
Obteniendo las capacidades de cada centro de acopio

In [7]:
# cap_df = pd.read_csv(csv_base + routes['cic'], **csv_config)
cap_df = pd.read_csv(csv_base + routes['cictest'], **csv_config)
cap_df

,CAcopios,Productos,CapInvCA
0,San Juan del Cesar,Queso duro,50
1,San Juan del Cesar,Queso blando,320
2,Riohacha,Queso duro,100
3,Riohacha,Queso blando,90
4,Maicao,Queso duro,10
5,Maicao,Queso blando,0


### Costos de Inventario
$$
  CInventario_t = \sum_{j \in J} \sum_{p \in P} CostoInvAcopio_{pjt} InvCA_{pjt} \qquad \forall_t \in T
$$

In [8]:
# cia_df = pd.read_csv(csv_base + routes['cia'], **csv_config)
cia_df = pd.read_csv(csv_base + routes['ciatest'], **csv_config)
cia_df.head()

,CAcopios,Productos,CostoInvCA
0,San Juan del Cesar,Queso duro,1000
1,San Juan del Cesar,Queso blando,1500
2,Riohacha,Queso duro,2000
3,Riohacha,Queso blando,800
4,Maicao,Queso duro,5000


### Costos de Transporte
$$
  CTransporte_t = \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} CostoTransAcopioClie_{pjkt} \qquad \forall_t \in T
$$

In [9]:
# ctac_df = pd.read_csv(csv_base + routes['ctac'], **csv_config)
ctac_df = pd.read_csv(csv_base + routes['ctactest'], **csv_config)
ctac_df.head()

,CAcopios,Clientes,CostoTransCAClie
0,San Juan del Cesar,Distribuidor1,5000
1,San Juan del Cesar,Distribuidor2,38000
2,Riohacha,Distribuidor1,5000
3,Riohacha,Distribuidor2,15000
4,Maicao,Distribuidor1,5000


## Esquema de representación
Se recibe la demanda $DemandaClie_{pkt}$ como parámetro de entrada.

Se representa la cantidad $n$ de centros de acopio como un vector, donde se asigna una cantidad $x_i$ para cada centro de acopio $j_i$, la cantidad asignada representa la demanda que va a ser suplida por ese centro de acopio.
$$
\begin{array} {|r|r|r|r|r|r|}
    \hline x_0 & x_1 & x_2 & x_3 & \cdots & x_n \\
    \hline
\end{array}
\quad \therefore \quad x_i = AC_{pjkt}
$$
Donde se aplican las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{n} x_i &= DemandaClie_{pkt} \\
    x_i &\leq InvCA_{pjt}
\end{align*}
$$

# Escenarios de prueba
Tomando demandas por cada centro de acopio, se definen distintos escenarios de posibles casos.

**Escenario 1**: Un centro de acopio sobrepasa la demanda del cliente
- El modelo puede asignar dicho centro de acopio para suplir con la demanda
- El centro de acopio debe proveer esa demanda

**Escenario 2**: Ningún centro de acopio cumple con toda la demanda del cliente
- El modelo debe dar una solución de varios centros de acopio
- La suma de la demanda por cada centro de acopio debe ser igual a la demanda

**Escenario 3**: Un centro de acopio tiene en stock exactamente la cantidad de la demanda de un cliente.
- El modelo puede tomar el centro de acopio que tiene en stock exactamente la demanda a cumplir

### Demandas
Generando demandas para cada cliente, se usarán como entrada para el modelo

In [27]:
demanda = 120
cap_queso = cap_df[cap_df[productos] == 'Queso duro']
bounds = tuple((0, x_i) for x_i in cap_queso['CapInvCA'])

## Función objetivo
$$
Min(F) = CInventario + CTransporte
$$

In [11]:
# F(x)
def f(x):
    delta = 0

    for idx in range(x.shape[0]):
        ca = cacopios_df.loc[idx]

        if x[idx] == 0.:
            continue

        # Costos de Inventario
        costo_inv_ca = cia_df[(cia_df[cacopios] == ca[cacopios]) & (cia_df[productos] == 'Queso duro')]
        # Costos de Transporte
        costo_trans_ca = ctac_df[(ctac_df[cacopios] == ca[cacopios]) & (ctac_df[clientes] == 'Distribuidor1')]

        delta += x[idx] * costo_inv_ca['CostoInvCA'].values[0]
        delta += np.array(costo_trans_ca['CostoTransCAClie'].values[0])
    
    return delta

## scipy

**LinearConstraint**: $ lb \leq A \cdot v[x] \leq ub $

**NonLinearConstraint**: $ lb \leq g(x) \leq ub $

In [55]:
c = tuple(idx for (_, idx) in bounds)
Aeq = np.ones((1, len(c)))
beq = 1

In [72]:
from scipy.optimize import linprog
# from scipy.optimize import differential_evolution

result = linprog(
    c,
    A_ub=Aeq,
    b_ub=beq,
    bounds=bounds,
    method='highs',
    integrality=1
)
print(result)

       message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
       success: True
        status: 0
           fun: 0.0
             x: [ 0.000e+00  0.000e+00  0.000e+00]
           nit: -1
         lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                marginals: [ 0.000e+00  0.000e+00  0.000e+00]
         upper:  residual: [ 5.000e+01  1.000e+02  1.000e+01]
                marginals: [ 0.000e+00  0.000e+00  0.000e+00]
         eqlin:  residual: []
                marginals: []
       ineqlin:  residual: [ 1.000e+00]
                marginals: [ 0.000e+00]


## pymoo

In [28]:
from pymoo.core.problem import ElementwiseProblem

xl = np.zeros(cacopios_df.shape)
xu = np.asarray([b for _, b in bounds])

class Queso(ElementwiseProblem):
    def __init__(self):
        super().__init__(
            n_var=len(xl),
            n_obj=1,
            # n_eq_constr=1,
            n_ieq_constr=1,
            xl=0,
            xu=xu,
            vtype=int
        )
        
    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = f(x)
        out['G'] = demanda - np.sum(x)
        # out['H'] = demanda - np.sum(x)

In [29]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.termination import get_termination

problem = Queso()

algorithm = GA(
    pop_size=9,
    eliminate_duplicates=True
)

termination = get_termination('time', '00:00:05')

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               verbose=False)

print(f'F: {res.F}, \nX: {res.X}')

F: [200187.39029581], 
X: [49.95628312 70.11555359  0.        ]


In [3]:
import pyomo.environ as pyo
import json

# Cargar datos desde el archivo JSON
json_file_path = './data/datos.json'
with open(json_file_path, 'r') as f:
    data = json.load(f)

# Extraer datos del JSON
demanda = data['demanda']
cTiempo = data['cTiempo']
centros_acopio = data['centros_acopio']
centro_principal = data['centro_acopio_principal']
tiempo_max_definido = data['tiempo_max_definido']

# Crear el modelo
model = pyo.ConcreteModel()

# Conjuntos
model.Centros = pyo.RangeSet(len(centros_acopio))

# Parámetros
model.demanda = pyo.Param(initialize=demanda)
model.cTiempo = pyo.Param(initialize=cTiempo)
model.tiempo_max = pyo.Param(initialize=tiempo_max_definido)

model.stock = pyo.Param(model.Centros, initialize={i+1: centros_acopio[i]['stock'] for i in range(len(centros_acopio))})
model.produccion_potencial = pyo.Param(model.Centros, initialize={i+1: centros_acopio[i]['produccion_potencial'] for i in range(len(centros_acopio))})
model.precio = pyo.Param(model.Centros, initialize={i+1: centros_acopio[i]['precio'] for i in range(len(centros_acopio))})
model.costo_transporte = pyo.Param(model.Centros, initialize={i+1: centros_acopio[i]['costo_transporte'] for i in range(len(centros_acopio))})
model.tiempo_alistamiento = pyo.Param(model.Centros, initialize={i+1: centros_acopio[i]['tiempo_alistamiento'] for i in range(len(centros_acopio))})
model.tiempo_transporte = pyo.Param(model.Centros, initialize={i+1: centros_acopio[i]['tiempo_transporte'] for i in range(len(centros_acopio))})

# Variables
model.x = pyo.Var(model.Centros, domain=pyo.NonNegativeReals)

# Restricciones
def demanda_total_rule(model):
    return sum(model.x[i] for i in model.Centros) == model.demanda

model.demanda_total = pyo.Constraint(rule=demanda_total_rule)

def restriccion_stock_rule(model, i):
    return model.x[i] <= model.stock[i]

model.restriccion_stock = pyo.Constraint(model.Centros, rule=restriccion_stock_rule)

def tiempo_total_rule(model):
    return sum((model.tiempo_alistamiento[i] + model.tiempo_transporte[i]) * model.x[i] for i in model.Centros) <= model.tiempo_max

model.tiempo_total = pyo.Constraint(rule=tiempo_total_rule)

# Función objetivo
def objetivo_rule(model):
    return sum(model.precio[i] * model.x[i] for i in model.Centros) + sum(model.costo_transporte[i] * model.x[i] for i in model.Centros)

model.objetivo = pyo.Objective(rule=objetivo_rule, sense=pyo.minimize)

# Resolver el modelo
solver = pyo.SolverFactory('glpk')
resultado = solver.solve(model)

# Mostrar resultados
print("Estado de la solución:", resultado.solver.status)
print("Estado de la optimización:", resultado.solver.termination_condition)
print("Demanda asignada a cada centro de acopio:")
for i in model.Centros:
    print(f"Centro {i}: {pyo.value(model.x[i])} unidades")

Estado de la solución: ok
Estado de la optimización: other
Demanda asignada a cada centro de acopio:
ERROR: evaluating object as numeric value: x[1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[1]


ValueError: No value for uninitialized NumericValue object x[1]

In [3]:
import pyomo.environ as pyo

# Crear el modelo
model = pyo.ConcreteModel()

# Parámetros
model.Demanda = pyo.Param(initialize=500)
model.Centros = pyo.RangeSet(1, 8)

# Datos de los centros de acopio (esto es solo un ejemplo, usa tus datos reales)
data = {
    1: {'stock': 100, 'produccion_potencial': 50, 'precio': 25, 'costo_transporte': 5, 'tiempo_alistamiento': 2, 'tiempo_transporte': 3},
    2: {'stock': 150, 'produccion_potencial': 30, 'precio': 26, 'costo_transporte': 6, 'tiempo_alistamiento': 1, 'tiempo_transporte': 4},
    3: {'stock': 120, 'produccion_potencial': 60, 'precio': 24, 'costo_transporte': 5, 'tiempo_alistamiento': 3, 'tiempo_transporte': 2},
    4: {'stock': 130, 'produccion_potencial': 40, 'precio': 27, 'costo_transporte': 7, 'tiempo_alistamiento': 2, 'tiempo_transporte': 3},
    5: {'stock': 90, 'produccion_potencial': 55, 'precio': 22, 'costo_transporte': 6, 'tiempo_alistamiento': 1, 'tiempo_transporte': 4},
    6: {'stock': 110, 'produccion_potencial': 45, 'precio': 28, 'costo_transporte': 5, 'tiempo_alistamiento': 3, 'tiempo_transporte': 3},
    7: {'stock': 140, 'produccion_potencial': 70, 'precio': 29, 'costo_transporte': 8, 'tiempo_alistamiento': 2, 'tiempo_transporte': 4},
    8: {'stock': 100, 'produccion_potencial': 50, 'precio': 23, 'costo_transporte': 6, 'tiempo_alistamiento': 1, 'tiempo_transporte': 2}
}

# Variables
model.x = pyo.Var(model.Centros, within=pyo.NonNegativeReals)

# Función objetivo (ejemplo simple de minimizar costos)
model.obj = pyo.Objective(expr=sum(model.x[i] * data[i]['precio'] for i in model.Centros), sense=pyo.minimize)

# Restricciones
def demanda_rule(model):
    return sum(model.x[i] for i in model.Centros) >= model.Demanda
model.DemandaTotal = pyo.Constraint(rule=demanda_rule)

def capacidad_rule(model, i):
    return model.x[i] <= data[i]['stock']
model.Capacidad = pyo.Constraint(model.Centros, rule=capacidad_rule)

# Resolver el modelo
solver = pyo.SolverFactory('glpk')
solver.solve(model, tee=True)

# Imprimir los resultados
print("Estado de la solución:", pyo.SolverFactory('glpk').solve(model).solver.status)
print("Estado de la optimización:", pyo.SolverFactory('glpk').solve(model).solver.termination_condition)
print("Demanda asignada a cada centro de acopio:")
for i in model.Centros:
    print(f"Centro {i}: {pyo.value(model.x[i])} unidades")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpcu68ns3c.glpk.raw --wglp /tmp/tmpxn3n7xjw.glpk.glp --cpxlp
 /tmp/tmplryjw9oy.pyomo.lp
Reading problem data from '/tmp/tmplryjw9oy.pyomo.lp'...
9 rows, 8 columns, 16 non-zeros
68 lines were read
Writing problem data to '/tmp/tmpxn3n7xjw.glpk.glp'...
53 lines were written
GLPK Simplex Optimizer 5.0
9 rows, 8 columns, 16 non-zeros
Preprocessing...
1 row, 8 columns, 8 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =   0.000000000e+00 inf =   5.000e+02 (1)
      4: obj =   1.279000000e+04 inf =   0.000e+00 (0)
*     8: obj =   1.200000000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40420 bytes)
Writing basic solution to '/tmp/tmpcu68ns3c.glpk.raw'...
26 lines were written
Estado de la solución: ok
Esta

In [8]:
import pyomo.environ as pyo

# Datos del problema
D = 500  # Demanda total
N = 9  # Número de centros de acopio (incluyendo el principal)
centro_principal = 0  # ID del centro de acopio principal
tiempo_max_definido = 24  # Tiempo máximo definido

# Datos de los centros de acopio (esto es solo un ejemplo, usa tus datos reales)
data = {
    0: {'stock': 200, 'produccion_potencial': 100, 'precio': 30, 'costo_transporte': {i: 4 for i in range(N)}, 'tiempo_alistamiento': 2, 'tiempo_transporte': {i: 2 for i in range(N)}},
    1: {'stock': 100, 'produccion_potencial': 50, 'precio': 25, 'costo_transporte': {i: 5 for i in range(N)}, 'tiempo_alistamiento': 2, 'tiempo_transporte': {i: 3 for i in range(N)}},
    2: {'stock': 150, 'produccion_potencial': 30, 'precio': 26, 'costo_transporte': {i: 6 for i in range(N)}, 'tiempo_alistamiento': 1, 'tiempo_transporte': {i: 4 for i in range(N)}},
    3: {'stock': 120, 'produccion_potencial': 60, 'precio': 24, 'costo_transporte': {i: 5 for i in range(N)}, 'tiempo_alistamiento': 3, 'tiempo_transporte': {i: 2 for i in range(N)}},
    4: {'stock': 130, 'produccion_potencial': 40, 'precio': 27, 'costo_transporte': {i: 7 for i in range(N)}, 'tiempo_alistamiento': 2, 'tiempo_transporte': {i: 3 for i in range(N)}},
    5: {'stock': 90, 'produccion_potencial': 55, 'precio': 22, 'costo_transporte': {i: 6 for i in range(N)}, 'tiempo_alistamiento': 1, 'tiempo_transporte': {i: 4 for i in range(N)}},
    6: {'stock': 110, 'produccion_potencial': 45, 'precio': 28, 'costo_transporte': {i: 5 for i in range(N)}, 'tiempo_alistamiento': 3, 'tiempo_transporte': {i: 3 for i in range(N)}},
    7: {'stock': 140, 'produccion_potencial': 70, 'precio': 29, 'costo_transporte': {i: 8 for i in range(N)}, 'tiempo_alistamiento': 2, 'tiempo_transporte': {i: 4 for i in range(N)}},
    8: {'stock': 100, 'produccion_potencial': 50, 'precio': 23, 'costo_transporte': {i: 6 for i in range(N)}, 'tiempo_alistamiento': 1, 'tiempo_transporte': {i: 2 for i in range(N)}}
}

# Crear el modelo
model = pyo.ConcreteModel()

# Conjuntos
model.Centros = pyo.RangeSet(0, N-1)

# Parámetros
model.Demanda = pyo.Param(initialize=D)
model.Stock = pyo.Param(model.Centros, initialize={i: data[i]['stock'] for i in range(N)})
model.ProduccionPotencial = pyo.Param(model.Centros, initialize={i: data[i]['produccion_potencial'] for i in range(N)})
model.Precio = pyo.Param(model.Centros, initialize={i: data[i]['precio'] for i in range(N)})
model.CostoTransporte = pyo.Param(model.Centros, model.Centros, initialize={(i, j): data[i]['costo_transporte'][j] for i in range(N) for j in range(N)})
model.TiempoAlistamiento = pyo.Param(model.Centros, initialize={i: data[i]['tiempo_alistamiento'] for i in range(N)})
model.TiempoTransporte = pyo.Param(model.Centros, model.Centros, initialize={(i, j): data[i]['tiempo_transporte'][j] for i in range(N) for j in range(N)})
model.TiempoMaxDefinido = pyo.Param(initialize=tiempo_max_definido)

# Variables
model.x = pyo.Var(model.Centros, within=pyo.NonNegativeReals)  # Cantidad asignada desde cada centro de acopio
model.y = pyo.Var(model.Centros, model.Centros, within=pyo.NonNegativeReals)  # Cantidad transportada desde cada centro de acopio

# Función objetivo: Minimizar el costo total de satisfacer la demanda
def objetivo(model):
    return sum(model.x[i] * model.Precio[i] for i in model.Centros) + \
           sum(model.y[i, j] * model.CostoTransporte[i, j] for i in model.Centros for j in model.Centros)
model.obj = pyo.Objective(rule=objetivo, sense=pyo.minimize)

# Restricciones
# La demanda total debe ser satisfecha
def demanda_total_rule(model):
    return sum(model.x[i] for i in model.Centros) >= model.Demanda
model.DemandaTotal = pyo.Constraint(rule=demanda_total_rule)

# La cantidad asignada desde cada centro de acopio no puede exceder su stock y producción potencial
def capacidad_rule(model, i):
    return model.x[i] <= model.Stock[i] + model.ProduccionPotencial[i]
model.Capacidad = pyo.Constraint(model.Centros, rule=capacidad_rule)

# Restricción de tiempo máximo definido
def tiempo_rule(model, i):
    return model.TiempoAlistamiento[i] + sum(model.y[j, i] * model.TiempoTransporte[j, i] for j in model.Centros) <= model.TiempoMaxDefinido
model.Tiempo = pyo.Constraint(model.Centros, rule=tiempo_rule)

# Resolver el modelo
solver = pyo.SolverFactory('glpk')
solver.solve(model, tee=True)

# Imprimir resultados
print("Estado de la solución:", pyo.SolverFactory('glpk').solve(model).solver.status)
print("Estado de la optimización:", pyo.SolverFactory('glpk').solve(model).solver.termination_condition)
print("Demanda asignada a cada centro de acopio:")
for i in model.Centros:
    print(f"Centro {i}: {pyo.value(model.x[i])} unidades")




GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpgnm2899v.glpk.raw --wglp /tmp/tmpphenya00.glpk.glp --cpxlp
 /tmp/tmpa90qeud_.pyomo.lp
Reading problem data from '/tmp/tmpa90qeud_.pyomo.lp'...
19 rows, 90 columns, 99 non-zeros
345 lines were read
Writing problem data to '/tmp/tmpphenya00.glpk.glp'...
320 lines were written
GLPK Simplex Optimizer 5.0
19 rows, 90 columns, 99 non-zeros
Preprocessing...
10 rows, 90 columns, 90 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  4.000e+00  ratio =  4.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 10
      0: obj =   0.000000000e+00 inf =   5.000e+02 (1)
      3: obj =   1.405000000e+04 inf =   0.000e+00 (0)
*     9: obj =   1.158500000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (85195 bytes)
Writing basic solution to '/tmp/tmpgnm2899v.glpk.raw'...
118 lines were written
Estado de la soluc

In [17]:
import pyomo.environ as pyo

# Datos de ejemplo
data = {
    0: {'costo_transporte': {0: 0, 1: 10, 2: 20}, 'tiempo_alistamiento': 5, 'tiempo_transporte': {0: 0, 1: 1, 2: 2}},
    1: {'costo_transporte': {0: 10, 1: 0, 2: 15}, 'tiempo_alistamiento': 6, 'tiempo_transporte': {0: 1, 1: 0, 2: 3}},
    2: {'costo_transporte': {0: 20, 1: 15, 2: 0}, 'tiempo_alistamiento': 7, 'tiempo_transporte': {0: 2, 1: 3, 2: 0}}
}

# Crear el modelo
model = pyo.ConcreteModel()

# Conjuntos
model.Centros = pyo.Set(initialize=data.keys())
model.Productos = pyo.Set(initialize=[0, 1, 2])  # Asumiendo productos numerados 0, 1 y 2

# Parámetros
model.CostoTransporte = pyo.Param(model.Centros, model.Centros, initialize=lambda model, i, j: data[i]['costo_transporte'][j])
model.TiempoAlistamiento = pyo.Param(model.Centros, initialize=lambda model, i: data[i]['tiempo_alistamiento'])
model.TiempoTransporte = pyo.Param(model.Centros, model.Centros, initialize=lambda model, i, j: data[i]['tiempo_transporte'][j])
model.TiempoMaxDefinido = pyo.Param(initialize=10)  # Valor de ejemplo

# Variables
model.x = pyo.Var(model.Centros, model.Productos, domain=pyo.Binary)  # Asignación de productos a centros
model.z = pyo.Var(model.Centros, domain=pyo.Binary)  # Variable para indicar el centro principal

# Función objetivo
def objetivo(model):
    return sum(model.CostoTransporte[i, j] * model.x[i, j] for i in model.Centros for j in model.Productos)
model.objetivo = pyo.Objective(rule=objetivo, sense=pyo.minimize)

# Restricción de tiempo
def restriccion_tiempo(model, i):
    return model.TiempoAlistamiento[i] + sum(model.TiempoTransporte[i, j] * model.x[i, j] for j in model.Productos) <= model.TiempoMaxDefinido
model.restriccion_tiempo = pyo.Constraint(model.Centros, rule=restriccion_tiempo)

# Restricción de un único centro principal
def restriccion_unico_principal(model):
    return sum(model.z[i] for i in model.Centros) == 1
model.restriccion_unico_principal = pyo.Constraint(rule=restriccion_unico_principal)

# Restricción de asignación de productos al centro principal
def restriccion_asignacion(model, i, j):
    return model.x[i, j] <= model.z[i]
model.restriccion_asignacion = pyo.Constraint(model.Centros, model.Productos, rule=restriccion_asignacion)

# Resolución del modelo
solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Mostrar resultados
print("Estado de la solución:", results.solver.status)
print("Estado de la optimización:", results.solver.termination_condition)

print("Demanda asignada a cada centro de acopio:")
for i in model.Centros:
    for j in model.Productos:
        if pyo.value(model.x[i, j]) > 0:
            print(f"Centro {i}, Producto {j}: {pyo.value(model.x[i, j])} unidades")

print("Centro principal:")
for i in model.Centros:
    if pyo.value(model.z[i]) > 0:
        print(f"Centro principal: {i}")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpbi6pkqmh.glpk.raw --wglp /tmp/tmpaf4qn6vl.glpk.glp --cpxlp
 /tmp/tmp4ztle1pz.pyomo.lp
Reading problem data from '/tmp/tmp4ztle1pz.pyomo.lp'...
/tmp/tmp4ztle1pz.pyomo.lp:94: warning: lower bound of variable 'x2' redefined
/tmp/tmp4ztle1pz.pyomo.lp:94: warning: upper bound of variable 'x2' redefined
13 rows, 12 columns, 27 non-zeros
12 integer variables, all of which are binary
106 lines were read
Writing problem data to '/tmp/tmpaf4qn6vl.glpk.glp'...
74 lines were written
GLPK Integer Optimizer 5.0
13 rows, 12 columns, 27 non-zeros
12 integer variables, all of which are binary
Preprocessing...
1 hidden packing inequaliti(es) were detected
11 rows, 12 columns, 23 non-zeros
12 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 11
Solvin

In [18]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Datos de entrada
data = {
    0: {'stock': 200, 'produccion_potencial': 100, 'precio': 30, 'costo_transporte': {i: 4 for i in range(9)}, 'tiempo_alistamiento': 2, 'tiempo_transporte': {i: 2 for i in range(9)}},
    1: {'stock': 100, 'produccion_potencial': 50, 'precio': 25, 'costo_transporte': {i: 5 for i in range(9)}, 'tiempo_alistamiento': 2, 'tiempo_transporte': {i: 3 for i in range(9)}},
    2: {'stock': 150, 'produccion_potencial': 30, 'precio': 26, 'costo_transporte': {i: 6 for i in range(9)}, 'tiempo_alistamiento': 1, 'tiempo_transporte': {i: 4 for i in range(9)}},
    3: {'stock': 120, 'produccion_potencial': 60, 'precio': 24, 'costo_transporte': {i: 5 for i in range(9)}, 'tiempo_alistamiento': 3, 'tiempo_transporte': {i: 2 for i in range(9)}},
    4: {'stock': 130, 'produccion_potencial': 40, 'precio': 27, 'costo_transporte': {i: 7 for i in range(9)}, 'tiempo_alistamiento': 2, 'tiempo_transporte': {i: 3 for i in range(9)}},
    5: {'stock': 90, 'produccion_potencial': 55, 'precio': 22, 'costo_transporte': {i: 6 for i in range(9)}, 'tiempo_alistamiento': 1, 'tiempo_transporte': {i: 4 for i in range(9)}},
    6: {'stock': 110, 'produccion_potencial': 45, 'precio': 28, 'costo_transporte': {i: 5 for i in range(9)}, 'tiempo_alistamiento': 3, 'tiempo_transporte': {i: 3 for i in range(9)}},
    7: {'stock': 140, 'produccion_potencial': 70, 'precio': 29, 'costo_transporte': {i: 8 for i in range(9)}, 'tiempo_alistamiento': 2, 'tiempo_transporte': {i: 4 for i in range(9)}},
    8: {'stock': 100, 'produccion_potencial': 50, 'precio': 23, 'costo_transporte': {i: 6 for i in range(9)}, 'tiempo_alistamiento': 1, 'tiempo_transporte': {i: 2 for i in range(9)}}
}

# Inicialización del modelo
model = pyo.ConcreteModel()

# Índices
model.Centros = pyo.RangeSet(0, 8)  # 9 centros de acopio
model.Productos = pyo.RangeSet(0, 8)  # 9 productos

# Variables
model.x = pyo.Var(model.Centros, domain=pyo.Binary)  # Variables binarias para elegir centro de acopio principal
model.y = pyo.Var(model.Centros, model.Productos, domain=pyo.NonNegativeReals)  # Cantidad de producto asignado a cada centro

# Parámetros
model.stock = pyo.Param(model.Centros, initialize={k: v['stock'] for k, v in data.items()})
model.produccion_potencial = pyo.Param(model.Centros, initialize={k: v['produccion_potencial'] for k, v in data.items()})
model.precio = pyo.Param(model.Centros, initialize={k: v['precio'] for k, v in data.items()})
model.costo_transporte = pyo.Param(model.Centros, model.Centros, initialize=lambda model, i, j: data[i]['costo_transporte'][j])
model.tiempo_alistamiento = pyo.Param(model.Centros, initialize={k: v['tiempo_alistamiento'] for k, v in data.items()})
model.tiempo_transporte = pyo.Param(model.Centros, model.Centros, initialize=lambda model, i, j: data[i]['tiempo_transporte'][j])

# Parámetros adicionales
model.TiempoMaxDefinido = pyo.Param(initialize=10)  # Tiempo máximo permitido para alistamiento y transporte

# Restricciones
def restriccion_tiempo(model, i):
    return model.tiempo_alistamiento[i] + sum(model.tiempo_transporte[i, j] * model.y[j, i] for j in model.Centros) <= model.TiempoMaxDefinido
model.restriccion_tiempo = pyo.Constraint(model.Centros, rule=restriccion_tiempo)

def restriccion_unico_principal(model):
    return sum(model.x[i] for i in model.Centros) == 1
model.restriccion_unico_principal = pyo.Constraint(rule=restriccion_unico_principal)

def restriccion_asignacion(model, i, j):
    return model.y[j, i] <= model.stock[j] * model.x[i]
model.restriccion_asignacion = pyo.Constraint(model.Centros, model.Productos, rule=restriccion_asignacion)

# Función objetivo
def objetivo(model):
    return sum(model.costo_transporte[i, j] * model.y[j, i] for i in model.Centros for j in model.Productos)
model.objetivo = pyo.Objective(rule=objetivo, sense=pyo.minimize)

# Resolver
solver = SolverFactory('glpk')
results = solver.solve(model)

# Imprimir resultados
print("Demanda asignada a cada centro de acopio:")
for i in model.Centros:
    print(f"Centro {i}: {pyo.value(model.x[i])} unidades")

print("Asignación de productos a centros de acopio:")
for i in model.Centros:
    for j in model.Productos:
        if pyo.value(model.y[j, i]) > 0:
            print(f"Producto {j} asignado al Centro {i}: {pyo.value(model.y[j, i])} unidades")

print("Costo total de transporte:", pyo.value(model.objetivo))


Demanda asignada a cada centro de acopio:
Centro 0: 0.0 unidades
Centro 1: 0.0 unidades
Centro 2: 0.0 unidades
Centro 3: 0.0 unidades
Centro 4: 0.0 unidades
Centro 5: 0.0 unidades
Centro 6: 0.0 unidades
Centro 7: 0.0 unidades
Centro 8: 1.0 unidades
Asignación de productos a centros de acopio:
Costo total de transporte: 0.0
